<a href="https://colab.research.google.com/github/mt508/machine-learning-/blob/main/heartattack_predict_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation, LeakyReLU

from tensorflow.keras import  layers, models
import matplotlib.pyplot as plt
from zlib import crc32
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [2]:
data=pd.read_csv("/content/Medicaldataset.csv")

In [3]:
data=data.drop(columns=["CK-MB","Troponin"])

In [4]:
import numpy as np
import pandas as pd
from zlib import crc32

# Example dataset
# data = pd.read_csv("your_file.csv")

# Make sure data is a DataFrame
print(type(data))

# Function definitions
def is_id_in_test_set(identifier, test_ratio):
    return crc32(np.int64(identifier)) < test_ratio * 2**32

def split_data_with_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: is_id_in_test_set(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

# Create artificial stable ID
data["id"] = np.arange(len(data))

# Make a copy (optional)
data_with_id = data.copy()

# Train-test split
train_set, test_set = split_data_with_id(data_with_id, 0.2, "id")

print(train_set.head())
print(test_set.head())


<class 'pandas.core.frame.DataFrame'>
   Age  Gender  Heart rate  Systolic blood pressure  Diastolic blood pressure  \
0   64       1          66                      160                        83   
1   21       1          94                       98                        46   
3   64       1          70                      120                        55   
4   55       1          64                      112                        65   
6   32       0          40                      179                        68   

   Blood sugar    Result  id  
0        160.0  negative   0  
1        296.0  positive   1  
3        270.0  positive   3  
4        300.0  negative   4  
6        102.0  negative   6  
    Age  Gender  Heart rate  Systolic blood pressure  \
2    55       1          64                      160   
5    58       0          61                      112   
12   64       1          60                      199   
16   86       0          73                      114   
23   30  

In [5]:
train_copy=train_set.copy()
X = train_copy.drop(columns=["Result","id"])
# Target
y = data["Result"]

# Encode target
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y)


In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X)


In [7]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled_train)


In [8]:
print(X_poly.shape)


(1055, 27)


In [9]:
import numpy as np
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))


{np.int64(0): np.int64(509), np.int64(1): np.int64(810)}

In [61]:
model = Sequential([

    Dense(32, activation='tanh', input_shape=(6,)),



    Dense(64, activation='tanh'),
    BatchNormalization(),

    Dense(128, activation='tanh'),



    Dense(1, activation='sigmoid')
])


In [40]:
model2= Sequential([

    Dense(32, activation='tanh', input_shape=(6,)),
    BatchNormalization(),
    Dropout(0.1),


    Dense(64, activation='tanh'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(1, activation='sigmoid')
])

In [31]:
from tensorflow.keras.optimizers import SGD

model2.compile(
    optimizer = SGD(
    learning_rate=0.01,    # must be tuned
    momentum=0.9,          # optional but recommended
    nesterov=True          # optional
),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [57]:
from tensorflow.keras.metrics import Precision

In [67]:
model.compile(
    optimizer = Adam(learning_rate=0.001, beta_1=0.7, beta_2=0.9),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision')]
)


In [65]:
es = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [49]:



history = model2.fit(
    X_scaled_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[es],
    verbose=1
)


ValueError: You must call `compile()` before using the model.

In [68]:
history = model.fit(
    X_scaled_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    callbacks=[es],
    verbose=1
)


Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.6034 - loss: 0.6726 - precision: 0.6257 - val_accuracy: 0.6132 - val_loss: 0.6518 - val_precision: 0.6111
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5958 - loss: 0.6706 - precision: 0.6086 - val_accuracy: 0.5755 - val_loss: 0.7017 - val_precision: 0.6000
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6181 - loss: 0.6609 - precision: 0.6370 - val_accuracy: 0.5943 - val_loss: 0.6584 - val_precision: 0.6000
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5999 - loss: 0.6640 - precision: 0.6161 - val_accuracy: 0.6132 - val_loss: 0.6527 - val_precision: 0.6064
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6236 - loss: 0.6493 - precision: 0.6328 - val_accuracy: 0.6226 - val_loss: 0.6740 - val_precision: 0.6180
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6833 - loss: 0.6294 - precision: 0.6787 - val_accuracy: 0.6038 - val_loss: 0.66